## Collecting LoL Comments from Reddit
Gulliver Wutz

In [2]:
import praw
import sys
import time
import datetime
import pytz
import time
import pandas as pd
from tqdm import tqdm

def unix_to_cest(unix_timestamp):
    dt_utc = datetime.datetime.utcfromtimestamp(unix_timestamp)
    cest = pytz.timezone('Europe/Berlin')
    dt_cest = dt_utc.replace(tzinfo=pytz.utc).astimezone(cest)
    formatted_dt = dt_cest.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_dt

pd.set_option('display.max_colwidth', 200)

In [3]:
reddit = praw.Reddit(
    client_id = "YOUR_ID",
    client_secret = "YOUR_SECRET",
    user_agent = "YOUR_APP_NAME"
)

In [7]:
# Set up Dataframe
columns = [
    "Parent ID", "Comment ID", "Author", "Score",
    "Created CEST", "Collected CEST", "Content"
]
comments_df = pd.DataFrame(columns=columns)
reddit_posts = pd.read_csv('C:/Users/wutzg/Desktop/Master Thesis/Artifacts/Datasets/reddit_posts_trimmed.csv')

In [8]:
# Use this to continue from a saved checkpoint
comments_df = pd.read_csv('reddit_comments.csv')
parent_ids = comments_df['Parent ID'].unique()
reddit_posts = reddit_posts[~reddit_posts['ID'].isin(parent_ids)]

In [9]:
iter = 0

for id in tqdm(reddit_posts[reddit_posts['Comment Count'] > 0]['ID'], desc="Processing"):
    iter +=1

    # Get the current Unix timestamp
    current_timestamp = int(time.time())
    comments = reddit.submission(id=id).comments

    for comment in comments:
        if hasattr(comment, 'id') and hasattr(comment, 'author') and hasattr(comment, 'score') and hasattr(comment, 'created_utc') and hasattr(comment, 'body'):
            new_data = {
            'Parent ID': id,
            'Comment ID': comment.id,
            'Author': comment.author,
            'Score': comment.score,
            'Created CEST': unix_to_cest(comment.created_utc),
            'Collected CEST': unix_to_cest(current_timestamp),
            'Content': comment.body
            }

            new_row = pd.DataFrame([new_data])
            comments_df = pd.concat([comments_df, new_row], ignore_index=True)
    
    # Save Checkpoint every 100 Posts
    if iter % 100 == 0:
        comments_df.to_csv('reddit_comments.csv', index=False)

    time.sleep(0.6)

comments_df.to_csv('reddit_comments.csv', index=False)

Processing: 100%|██████████| 2501/2501 [42:43<00:00,  1.02s/it] 


In [16]:
# Load CSV conatining all collected comments
reddit_comments = pd.read_csv('reddit_comments.csv')
print("Dataset size: " + str(reddit_comments.shape))
print("\n")


if reddit_comments['Parent ID'].is_unique:
    print("Parent IDs are unique!")
else:
    print("Parent IDs are NOT unique!")

if reddit_comments['Comment ID'].is_unique:
    print("Post Full IDs are unique!")
else:
    print("Post Full IDs NOT are unique!")

if reddit_comments.isna().any().any():
    print(str(reddit_comments.isna().sum().sum()) + " NaN-Values detected!")
else:
    print("No NaN-Values detected!")

reddit_comments

Dataset size: (329088, 7)


Parent IDs are NOT unique!
Post Full IDs are unique!
6638 NaN-Values detected!


,Parent ID,Comment ID,Author,Score,Created CEST,Collected CEST,Content
0,1ch5547,l20835j,Responsible-Jury8618,77,2024-05-01 00:07:40,2024-05-27 13:25:36,"Ok, this is HUGE, all of Seraphine's dmg is AOE and old liandry was amazing because of that, this looks like a new first item for mid/apc\n\nIt gives all of the stats we need for a first item with..."
1,1ch5547,l2087qq,London_Tipton,29,2024-05-01 00:08:28,2024-05-27 13:25:36,Definitely core for carry Seraphine. It's basically mana liandry
2,1ch5547,l20730g,PromotionOdd8700,19,2024-05-01 00:01:29,2024-05-27 13:25:36,This item + liandrys looks like new core items??
3,1ch5547,l2098yb,Alive-Pop6243,15,2024-05-01 00:14:52,2024-05-27 13:25:36,Seraphine gonna love this plus Morgan’s jungle and Zyra jungle too also brand
4,1ch5547,l21dqtk,sum-dude,9,2024-05-01 04:36:25,2024-05-27 13:25:36,Now just give us back our 5 AD and we're good.
...,...,...,...,...,...,...,...
329083,192skjk,kjbpdj5,Amy_Sery,1,2024-01-24 09:31:32,2024-05-28 11:06:02,"#Heya everyone! The new testing cycle will begin very soon. Riot is no longer collecting/considering feedback on the current cycle and its content, which means that this thread is **now closed**. ..."
329084,192sicz,kh4rsn5,TannerCook100,3,2024-01-10 01:00:30,2024-05-28 11:06:06,"What you’re supposed to do changes depending on the stage of the game and the team comps, but the reality is that you can’t always do anything.\n\nEarly on, you say their Talon was terrible and yo..."
329085,192sicz,kh4sq1d,xdlol11,2,2024-01-10 01:06:19,2024-05-28 11:06:06,"We can't see the game, but in general there's usually a path to winning most games, someone much better than you would have maybe grouped at different times, played the map differently, that would..."
329086,192sicz,kh7eocl,RusticApothecary,1,2024-01-10 14:48:00,2024-05-28 11:06:06,"Flame them, spam FF and do a lot worse to show them who's boss 👍"
